I've downloaded a British and American corpus for training data for my word2vec model that will translate between the languages. The challenge will be getting them into a database, which is what I'll document here.

The British language is in a series of XML documents. I'm going to use Python to parse this and put the data into a SQL database.

In [3]:
import glob
import re, os
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

In [42]:
# Define a database name
# Set your postgres username
dbname = 'corpus'
username = 'dan' # change this to your username

In [16]:
from sqlalchemy import create_engine
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://dan@localhost/corpus


In [17]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


In [36]:
#Create table in database
command = (
        """
        CREATE TABLE training_data (
            source VARCHAR(20),
            sentence TEXT
            );
        """)
print(command)


        CREATE TABLE training_data (
            source VARCHAR(20),
            sentence TEXT
            );
        


In [37]:
conn = None
conn = psycopg2.connect(database = dbname, user = username)
cur = conn.cursor()

# create table one by one
cur.execute(command)
# close communication with the PostgreSQL database server
cur.close()
# commit the changes
conn.commit()

In [38]:
#Check to see if tables exist in db
cur = conn.cursor()
cur.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
print (cur.fetchall())

[('training_data',)]


In [39]:
#conn = psycopg2.connect(database = dbname, user = username)
#cur = conn.cursor()
#cur.execute('rollback;')# DROP TABLE training_data;')

In [40]:
def write_to_db(source, sentence):
    cur = conn.cursor()
    add_to_db = """INSERT INTO training_data(source,sentence) VALUES(%s,%s)"""
    data = (source,sentence.lower())
    cur.execute(add_to_db,data)
    cur.close()

In [41]:
# The British National Corpus has some XML files. I couldn't get BeautifulSoup to parse it properly, so instead
# I'm parsing them here by searching for the necessary tags and creating the sentences.

xml_files = glob.glob('British_National_Corpus/2554/download/Texts/**/**/*xml')

for file in xml_files:
    print(file)
    with open(file, 'r') as xml_file:
        xml_content = xml_file.read()
        xml_content = ''.join(xml_content)
        xml_content = xml_content.replace('<w c5="', '\n<w c5="').replace('<c c5="', '\n<c c5="')

        temp_sentence = ''
        for line in xml_content.split('\n'):
            if line[0:5] == '<w c5':
                word = line.split('>')[1].split('<')[0] # extract the word from the XML tag
                temp_sentence += word + ' '
            elif 'pause' not in line and len(temp_sentence) > 0:
                write_to_db('BNC', temp_sentence.replace(" '", "'"))
                #print(temp_sentence.replace(" '", "'"))
                temp_sentence = ''
    

#cur.close()
#conn.commit()

British_National_Corpus/2554/download/Texts/G/G6/G61.xml
British_National_Corpus/2554/download/Texts/G/G6/G60.xml
British_National_Corpus/2554/download/Texts/G/G6/G62.xml
British_National_Corpus/2554/download/Texts/G/G6/G63.xml
British_National_Corpus/2554/download/Texts/G/G6/G64.xml
British_National_Corpus/2554/download/Texts/G/G1/G1H.xml
British_National_Corpus/2554/download/Texts/G/G1/G1J.xml
British_National_Corpus/2554/download/Texts/G/G1/G19.xml
British_National_Corpus/2554/download/Texts/G/G1/G1X.xml
British_National_Corpus/2554/download/Texts/G/G1/G1Y.xml
British_National_Corpus/2554/download/Texts/G/G1/G1N.xml
British_National_Corpus/2554/download/Texts/G/G1/G1L.xml
British_National_Corpus/2554/download/Texts/G/G1/G1M.xml
British_National_Corpus/2554/download/Texts/G/G1/G16.xml
British_National_Corpus/2554/download/Texts/G/G1/G1W.xml
British_National_Corpus/2554/download/Texts/G/G1/G1V.xml
British_National_Corpus/2554/download/Texts/G/G1/G17.xml
British_National_Corpus/2554/do

British_National_Corpus/2554/download/Texts/G/GY/GYB.xml
British_National_Corpus/2554/download/Texts/G/GY/GYC.xml
British_National_Corpus/2554/download/Texts/G/GY/GY5.xml
British_National_Corpus/2554/download/Texts/G/GY/GYT.xml
British_National_Corpus/2554/download/Texts/G/GY/GYV.xml
British_National_Corpus/2554/download/Texts/G/GY/GY7.xml
British_National_Corpus/2554/download/Texts/G/GY/GYA.xml
British_National_Corpus/2554/download/Texts/G/GY/GY6.xml
British_National_Corpus/2554/download/Texts/G/GY/GYW.xml
British_National_Corpus/2554/download/Texts/G/GY/GYD.xml
British_National_Corpus/2554/download/Texts/G/GY/GYS.xml
British_National_Corpus/2554/download/Texts/G/GY/GYR.xml
British_National_Corpus/2554/download/Texts/G/GY/GYE.xml
British_National_Corpus/2554/download/Texts/G/GY/GYG.xml
British_National_Corpus/2554/download/Texts/G/GY/GYP.xml
British_National_Corpus/2554/download/Texts/G/GY/GYF.xml
British_National_Corpus/2554/download/Texts/G/G5/G5B.xml
British_National_Corpus/2554/do

British_National_Corpus/2554/download/Texts/G/G3/G37.xml
British_National_Corpus/2554/download/Texts/G/G3/G3V.xml
British_National_Corpus/2554/download/Texts/G/G3/G3W.xml
British_National_Corpus/2554/download/Texts/G/G3/G36.xml
British_National_Corpus/2554/download/Texts/G/G3/G3B.xml
British_National_Corpus/2554/download/Texts/G/G3/G34.xml
British_National_Corpus/2554/download/Texts/G/G3/G3U.xml
British_National_Corpus/2554/download/Texts/G/G3/G3T.xml
British_National_Corpus/2554/download/Texts/G/G3/G35.xml
British_National_Corpus/2554/download/Texts/G/G3/G3C.xml
British_National_Corpus/2554/download/Texts/G/G3/G31.xml
British_National_Corpus/2554/download/Texts/G/G3/G3P.xml
British_National_Corpus/2554/download/Texts/G/G3/G3G.xml
British_National_Corpus/2554/download/Texts/G/G3/G3F.xml
British_National_Corpus/2554/download/Texts/G/G3/G30.xml
British_National_Corpus/2554/download/Texts/G/G3/G32.xml
British_National_Corpus/2554/download/Texts/G/G3/G3S.xml
British_National_Corpus/2554/do

British_National_Corpus/2554/download/Texts/A/AM/AMN.xml
British_National_Corpus/2554/download/Texts/A/AM/AMX.xml
British_National_Corpus/2554/download/Texts/A/AM/AM9.xml
British_National_Corpus/2554/download/Texts/A/AM/AMM.xml
British_National_Corpus/2554/download/Texts/A/AM/AML.xml
British_National_Corpus/2554/download/Texts/A/AM/AM7.xml
British_National_Corpus/2554/download/Texts/A/AM/AMA.xml
British_National_Corpus/2554/download/Texts/A/AM/AMW.xml
British_National_Corpus/2554/download/Texts/A/AM/AM6.xml
British_National_Corpus/2554/download/Texts/A/AM/AM4.xml
British_National_Corpus/2554/download/Texts/A/AM/AMU.xml
British_National_Corpus/2554/download/Texts/A/AM/AMB.xml
British_National_Corpus/2554/download/Texts/A/AM/AMC.xml
British_National_Corpus/2554/download/Texts/A/AM/AMT.xml
British_National_Corpus/2554/download/Texts/A/AM/AM5.xml
British_National_Corpus/2554/download/Texts/A/AM/AMG.xml
British_National_Corpus/2554/download/Texts/A/AM/AM1.xml
British_National_Corpus/2554/do

British_National_Corpus/2554/download/Texts/A/AY/AYJ.xml
British_National_Corpus/2554/download/Texts/A/AY/AYR.xml
British_National_Corpus/2554/download/Texts/A/AY/AYP.xml
British_National_Corpus/2554/download/Texts/A/AK/AKK.xml
British_National_Corpus/2554/download/Texts/A/AK/AKJ.xml
British_National_Corpus/2554/download/Texts/A/AK/AKH.xml
British_National_Corpus/2554/download/Texts/A/AK/AKM.xml
British_National_Corpus/2554/download/Texts/A/AK/AKL.xml
British_National_Corpus/2554/download/Texts/A/AK/AK8.xml
British_National_Corpus/2554/download/Texts/A/AK/AKY.xml
British_National_Corpus/2554/download/Texts/A/AK/AKN.xml
British_National_Corpus/2554/download/Texts/A/AK/AKX.xml
British_National_Corpus/2554/download/Texts/A/AK/AK9.xml
British_National_Corpus/2554/download/Texts/A/AK/AK4.xml
British_National_Corpus/2554/download/Texts/A/AK/AKU.xml
British_National_Corpus/2554/download/Texts/A/AK/AKB.xml
British_National_Corpus/2554/download/Texts/A/AK/AKC.xml
British_National_Corpus/2554/do

...

British_National_Corpus/2554/download/Texts/B/B1/B1E.xml
British_National_Corpus/2554/download/Texts/B/B1/B17.xml
British_National_Corpus/2554/download/Texts/B/B1/B1A.xml
British_National_Corpus/2554/download/Texts/B/B1/B1W.xml
British_National_Corpus/2554/download/Texts/B/B1/B16.xml
British_National_Corpus/2554/download/Texts/B/B1/B14.xml
British_National_Corpus/2554/download/Texts/B/B1/B1U.xml
British_National_Corpus/2554/download/Texts/B/B1/B1C.xml
British_National_Corpus/2554/download/Texts/B/B1/B1T.xml
British_National_Corpus/2554/download/Texts/B/B1/B15.xml
British_National_Corpus/2554/download/Texts/B/BN/BN3.xml
British_National_Corpus/2554/download/Texts/B/BN/BNE.xml
British_National_Corpus/2554/download/Texts/B/BN/BND.xml
British_National_Corpus/2554/download/Texts/B/BN/BNS.xml
British_National_Corpus/2554/download/Texts/B/BN/BN2.xml
British_National_Corpus/2554/download/Texts/B/BN/BNF.xml
British_National_Corpus/2554/download/Texts/B/BN/BNG.xml
British_National_Corpus/2554/do

British_National_Corpus/2554/download/Texts/K/KC/KCV.xml
British_National_Corpus/2554/download/Texts/K/KC/KCW.xml
British_National_Corpus/2554/download/Texts/K/KC/KC6.xml
British_National_Corpus/2554/download/Texts/K/KC/KCB.xml
British_National_Corpus/2554/download/Texts/K/KC/KC4.xml
British_National_Corpus/2554/download/Texts/K/KC/KCU.xml
British_National_Corpus/2554/download/Texts/K/KC/KCT.xml
British_National_Corpus/2554/download/Texts/K/KC/KC5.xml
British_National_Corpus/2554/download/Texts/K/KC/KCC.xml
British_National_Corpus/2554/download/Texts/K/KC/KC1.xml
British_National_Corpus/2554/download/Texts/K/KC/KCP.xml
British_National_Corpus/2554/download/Texts/K/KC/KCG.xml
British_National_Corpus/2554/download/Texts/K/KC/KCF.xml
British_National_Corpus/2554/download/Texts/K/KC/KC0.xml
British_National_Corpus/2554/download/Texts/K/KC/KC2.xml
British_National_Corpus/2554/download/Texts/K/KC/KCS.xml
British_National_Corpus/2554/download/Texts/K/KC/KCD.xml
British_National_Corpus/2554/do

British_National_Corpus/2554/download/Texts/K/K5/K5G.xml
British_National_Corpus/2554/download/Texts/K/K5/K51.xml
British_National_Corpus/2554/download/Texts/K/K5/K5P.xml
British_National_Corpus/2554/download/Texts/K/K5/K5R.xml
British_National_Corpus/2554/download/Texts/K/K5/K53.xml
British_National_Corpus/2554/download/Texts/K/K5/K5E.xml
British_National_Corpus/2554/download/Texts/K/K5/K5D.xml
British_National_Corpus/2554/download/Texts/K/K5/K52.xml
British_National_Corpus/2554/download/Texts/K/K5/K5S.xml
British_National_Corpus/2554/download/Texts/K/K5/K5W.xml
British_National_Corpus/2554/download/Texts/K/K5/K56.xml
British_National_Corpus/2554/download/Texts/K/K5/K57.xml
British_National_Corpus/2554/download/Texts/K/K5/K5V.xml
British_National_Corpus/2554/download/Texts/K/K5/K5A.xml
British_National_Corpus/2554/download/Texts/K/K5/K5C.xml
British_National_Corpus/2554/download/Texts/K/K5/K5T.xml
British_National_Corpus/2554/download/Texts/K/K5/K55.xml
British_National_Corpus/2554/do

British_National_Corpus/2554/download/Texts/K/K7/K77.xml
British_National_Corpus/2554/download/Texts/K/K7/K76.xml
British_National_Corpus/2554/download/Texts/K/K7/K74.xml
British_National_Corpus/2554/download/Texts/K/K7/K75.xml
British_National_Corpus/2554/download/Texts/K/K7/K78.xml
British_National_Corpus/2554/download/Texts/K/K7/K79.xml
British_National_Corpus/2554/download/Texts/K/KN/KN9.xml
British_National_Corpus/2554/download/Texts/K/KN/KN8.xml
British_National_Corpus/2554/download/Texts/K/KN/KNY.xml
British_National_Corpus/2554/download/Texts/K/KN/KNH.xml
British_National_Corpus/2554/download/Texts/K/KN/KNJ.xml
British_National_Corpus/2554/download/Texts/K/KN/KNK.xml
British_National_Corpus/2554/download/Texts/K/KN/KNF.xml
British_National_Corpus/2554/download/Texts/K/KN/KN1.xml
British_National_Corpus/2554/download/Texts/K/KN/KNG.xml
British_National_Corpus/2554/download/Texts/K/KN/KNR.xml
British_National_Corpus/2554/download/Texts/K/KN/KN3.xml
British_National_Corpus/2554/do

British_National_Corpus/2554/download/Texts/K/KA/KAK.xml
British_National_Corpus/2554/download/Texts/K/KA/KA9.xml
British_National_Corpus/2554/download/Texts/K/KA/KAX.xml
British_National_Corpus/2554/download/Texts/K/KA/KAY.xml
British_National_Corpus/2554/download/Texts/K/KA/KA8.xml
British_National_Corpus/2554/download/Texts/K/KA/KAN.xml
British_National_Corpus/2554/download/Texts/K/KA/KAL.xml
British_National_Corpus/2554/download/Texts/K/KA/KAM.xml


In [43]:
# This is a function that someone else wrote to split a big block of text into sentences.

caps = "([A-Z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + caps + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(caps + "[.]" + caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + caps + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences


conn = None
conn = psycopg2.connect(database = dbname, user = username)
cur = conn.cursor()

with os.popen('find *txt ./OANC-GrAF | grep "txt"$', 'r') as list_of_files:
    for filename in list_of_files:
        if 'OANC' in filename:
            print(filename)
            text = open(filename.strip(), 'r').read()
            list_of_sentences = split_into_sentences(text)
            for sentence in list_of_sentences:
                add_to_db = """INSERT INTO training_data(source,sentence) VALUES(%s,%s)"""
                data = ('OANC',sentence.lower())
                cur.execute(add_to_db,data)



    
    

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_28629.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_25319.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_27730.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_27917.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_21209.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_21547.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_22728.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_21553.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_29696.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_29331.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_27452.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_27446.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_23392.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_21234.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_28210.txt

./OANC-GrAF/data/written_

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_25632.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_21865.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_28896.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_29236.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_28119.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_21868.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_22622.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_22187.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_22756.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_26054.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_22554.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_23676.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_23663.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_25560.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_29400.txt

./OANC-GrAF/data/written_

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_22243.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_28008.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_27449.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_22280.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_21944.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_23174.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_23389.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_25934.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_21213.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_25713.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_26391.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_26187.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_20521.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_20251.txt

./OANC-GrAF/data/written_1/journal/slate/50/ArticleIP_22678.txt

./OANC-GrAF/data/written_

./OANC-GrAF/data/written_1/journal/slate/35/ArticleIP_2380.txt

./OANC-GrAF/data/written_1/journal/slate/35/ArticleIP_2396.txt

./OANC-GrAF/data/written_1/journal/slate/35/ArticleIP_2369.txt

./OANC-GrAF/data/written_1/journal/slate/35/ArticleIP_2368.txt

./OANC-GrAF/data/written_1/journal/slate/35/ArticleIP_2383.txt

./OANC-GrAF/data/written_1/journal/slate/35/ArticleIP_2397.txt

./OANC-GrAF/data/written_1/journal/slate/35/ArticleIP_2371.txt

./OANC-GrAF/data/written_1/journal/slate/35/ArticleIP_2365.txt

./OANC-GrAF/data/written_1/journal/slate/35/ArticleIP_2364.txt

./OANC-GrAF/data/written_1/journal/slate/35/ArticleIP_2370.txt

./OANC-GrAF/data/written_1/journal/slate/35/ArticleIP_2399.txt

./OANC-GrAF/data/written_1/journal/slate/35/ArticleIP_2373.txt

./OANC-GrAF/data/written_1/journal/slate/35/ArticleIP_2398.txt

./OANC-GrAF/data/written_1/journal/slate/35/ArticleIP_2388.txt

./OANC-GrAF/data/written_1/journal/slate/35/ArticleIP_2363.txt

./OANC-GrAF/data/written_1/journal/slate

./OANC-GrAF/data/written_1/journal/slate/51/ArticleIP_34172.txt

./OANC-GrAF/data/written_1/journal/slate/51/ArticleIP_35481.txt

./OANC-GrAF/data/written_1/journal/slate/51/ArticleIP_38172.txt

./OANC-GrAF/data/written_1/journal/slate/51/ArticleIP_33741.txt

./OANC-GrAF/data/written_1/journal/slate/51/ArticleIP_33580.txt

./OANC-GrAF/data/written_1/journal/slate/51/ArticleIP_32113.txt

./OANC-GrAF/data/written_1/journal/slate/51/ArticleIP_33225.txt

./OANC-GrAF/data/written_1/journal/slate/51/ArticleIP_34505.txt

./OANC-GrAF/data/written_1/journal/slate/51/ArticleIP_31496.txt

./OANC-GrAF/data/written_1/journal/slate/51/ArticleIP_32362.txt

./OANC-GrAF/data/written_1/journal/slate/51/ArticleIP_32438.txt

./OANC-GrAF/data/written_1/journal/slate/51/ArticleIP_37597.txt

./OANC-GrAF/data/written_1/journal/slate/51/ArticleIP_33108.txt

./OANC-GrAF/data/written_1/journal/slate/51/ArticleIP_32000.txt

./OANC-GrAF/data/written_1/journal/slate/51/ArticleIP_35790.txt

./OANC-GrAF/data/written_

...

./OANC-GrAF/data/written_2/technical/biomed/1471-2172-3-10.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2172-2-4.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2180-1-12.txt

./OANC-GrAF/data/written_2/technical/biomed/gb-2001-2-6-research0018.txt

./OANC-GrAF/data/written_2/technical/biomed/gb-2001-2-9-research0037.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2105-2-9.txt

./OANC-GrAF/data/written_2/technical/biomed/1472-6904-3-1.txt

./OANC-GrAF/data/written_2/technical/biomed/1472-6793-2-16.txt

./OANC-GrAF/data/written_2/technical/biomed/1472-6807-3-2.txt

./OANC-GrAF/data/written_2/technical/biomed/1476-072X-2-4.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2121-2-18.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2148-2-8.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2105-4-24.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2156-3-3.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2466-2-3.txt

./OANC-GrAF/data/written_2/t

./OANC-GrAF/data/written_2/technical/biomed/1472-6823-2-2.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2180-1-29.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2172-2-3.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2407-1-6.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2091-2-9.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2202-2-15.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2091-3-23.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2164-4-24.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-213X-1-4.txt

./OANC-GrAF/data/written_2/technical/biomed/gb-2002-3-12-research0080.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2180-3-10.txt

./OANC-GrAF/data/written_2/technical/biomed/cc1476.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2407-3-4.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2431-3-6.txt

./OANC-GrAF/data/written_2/technical/biomed/bcr294.txt

./OANC-GrAF/data/written_2/technical/biomed/gb-2002-

./OANC-GrAF/data/written_2/technical/biomed/1471-2091-3-14.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-5945-2-13.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2164-3-32.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2164-3-26.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2288-2-11.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2180-3-4.txt

./OANC-GrAF/data/written_2/technical/biomed/1472-6750-1-6.txt

./OANC-GrAF/data/written_2/technical/biomed/gb-2003-4-6-r39.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2458-2-25.txt

./OANC-GrAF/data/written_2/technical/biomed/gb-2003-4-3-r20.txt

./OANC-GrAF/data/written_2/technical/biomed/gb-2003-4-9-r57.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2121-3-2.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2407-2-23.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2105-4-28.txt

./OANC-GrAF/data/written_2/technical/biomed/1472-6947-2-7.txt

./OANC-GrAF/data/written_2/technical/biom

./OANC-GrAF/data/written_2/technical/biomed/1471-2229-2-8.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2474-4-8.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2121-3-18.txt

./OANC-GrAF/data/written_2/technical/biomed/1472-6882-1-11.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2334-3-13.txt

./OANC-GrAF/data/written_2/technical/biomed/cvm-2-1-038.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2121-3-30.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2156-4-10.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2199-2-3.txt

./OANC-GrAF/data/written_2/technical/biomed/1476-4598-1-3.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2172-2-10.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2121-2-6.txt

./OANC-GrAF/data/written_2/technical/biomed/1477-7827-1-21.txt

./OANC-GrAF/data/written_2/technical/biomed/1477-7827-1-23.txt

./OANC-GrAF/data/written_2/technical/biomed/1475-2891-1-2.txt

./OANC-GrAF/data/written_2/technical/biomed/1468-


./OANC-GrAF/data/written_2/technical/biomed/1471-2210-1-2.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2458-1-9.txt

./OANC-GrAF/data/written_2/technical/biomed/1472-6793-2-2.txt

./OANC-GrAF/data/written_2/technical/biomed/gb-2001-2-12-research0053.txt

./OANC-GrAF/data/written_2/technical/biomed/1478-1336-1-2.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2202-3-16.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2180-2-13.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2121-4-4.txt

./OANC-GrAF/data/written_2/technical/biomed/gb-2003-4-4-r28.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-230X-2-17.txt

./OANC-GrAF/data/written_2/technical/biomed/1477-5956-1-1.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2156-4-9.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2431-2-12.txt

./OANC-GrAF/data/written_2/technical/biomed/ar328.txt

./OANC-GrAF/data/written_2/technical/biomed/1471-2210-3-1.txt

./OANC-GrAF/data/written_2/technical/biomed/

Now the data are in Postgresql.